In [ ]:
#@title 設定
# ── 初期設定セル ──
# Colab の自動リロード機能（自作モジュール編集を即時反映）
%load_ext autoreload
%autoreload 2

# GitHub リポジトリをクローン（すでにクローン済みならスキップ可）
import os
if not os.path.isdir("/content/human-in-the-loop-transcribe"):
    !git clone https://github.com/khkata/human-in-the-loop-transcribe.git

# リポジトリ内に移動
%cd /content/human-in-the-loop-transcribe

# editable install ＆ Python 依存関係
!pip install -q -e .
!pip install -q -r requirements.txt

# システム依存パッケージ（MeCab など）
!apt-get update -qq && apt-get install -qq mecab libmecab-dev mecab-ipadic-utf8

# Google Drive マウント
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# パス設定
import sys
sys.path.append('/content/human-in-the-loop-transcribe')

# 環境チェック
import torch, whisper
print(f"Python: {sys.version.split()[0]}, torch: {torch.__version__}, GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

In [ ]:
#@title 特徴量抽出

!python src/extract_feats.py \
    --train_raw_csv $TRAIN_RAW_CSV \
    --train_processed_csv $TRAIN_PROCESSED_CSV \
    --train_raw_audio $TRAIN_RAW_AUDIO \
    --train_features $TRAIN_FEATURES \
    --test_raw_csv $TEST_RAW_CSV \
    --test_features $TEST_FEATURES

In [ ]:
#@title 訓練

!python src/train.py --seed 42 --trials 30

from src.train import retrain_full_data
retrain_full_data()

In [ ]:
#@title 評価

!python src/evaluate.py \
  --test_features $TEST_FEATURES \
  --scaler_path   $SCALER_PATH \
  --model_path    $MODEL_PATH \
  --processed_csv $TEST_PROCESSED_CSV

In [ ]:
#@title ROVER-3

!python src/integrate.py \
  --human_csv      $TEST_HUMAN_CSV \
  --processed_csv  $TEST_PROCESSED_CSV \
  --integrated_csv $TEST_INTEGRATED_CSV

In [ ]:
#@title Cost-WER Curve

!python src/cost_wer_curve.py \
  --integrated_csv $TEST_INTEGRATED_CSV \
  --human_cost 15.0 \
  --output_dir /content/drive/MyDrive/.../outputs \
  --output_filename cost_wer_curve.png